# Introduction
My assignment submission will chose Dublin, Ireland and perform postcode clustering to determine which post codes are the most similar. This will enable individuals looking to move house to chose the most appropriate locations.

# Data
The data I will use is a list of Dublin's postcodes combined with their latitude and longitude coordinates.  This dataset didn't exist in the public domain so was manually made using wikipedia to get a list of the postcodes in Dublin, these postcodes were then each manually Googled to return the latitude and longitude coorinates for each postcode. Due to an error with the manually created csv a null column had to be droped.

In [1]:
import pandas as pd

DublinData = pd.read_csv("CourseraDublinDistricts.csv", encoding = "ISO-8859-1")
DublinData= DublinData.drop([DublinData.columns[1]], axis='columns')
DublinData.head()

,District,Postcode,Latitude,Longitude
0,Dublin,Dublin 1,53.35,-6.26
1,Dublin,Dublin 2,53.33,-6.25
2,Dublin,Dublin 3,53.36,-6.22
3,Dun Laoghaire Rathdown,Dublin 4,53.32,-6.23
4,Dublin,Dublin 5,53.38,-6.19


In total we have 22 postcodes for Dublin as per the below code  output.

In [2]:
DublinData.shape

(22, 4)

Visualise the postcodes

In [3]:
import folium

from folium.plugins import FastMarkerCluster, MarkerCluster
latitude = 53.34
longitude = -6.25
# create map of New York using latitude and longitude values
map_Dublin = folium.Map(location=[latitude, longitude], zoom_start=10)
mc = MarkerCluster(name="Marker Cluster")

for index, row in DublinData.iterrows():
    popup_text = "{}".format(row["Postcode"])
    folium.CircleMarker(location=[row["Latitude"],row["Longitude"]],
                        radius= 10,
                        color="red",
                        popup=popup_text,
                        fill=True).add_to(mc)

mc.add_to(map_Dublin)

folium.LayerControl().add_to(map_Dublin)

    
map_Dublin

Create Foursquare handshake token.

In [4]:
# add markers to map
CLIENT_ID = '5UKMQFKKSXE5LCX4KOQC5CJ5WSBFFWL4ESOXPODP4QDCFR3U'
CLIENT_SECRET = 'GDC244JSZK1WYLBORYNLVBAJHQXGJPNLAFGRILKHFKGQWSMB'
VERSION = '20190605'

Pull in the venue details for the locations provided using the latitude and longitude coordinates. Store the venues in a list.

In [5]:
import requests 
LIMIT =  100
venues_list=[]
radius =1500

for name,lat,lng in zip(DublinData['Postcode'], DublinData['Latitude'], DublinData['Longitude']):
    url = 'http://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)
    results = requests.get(url).json()["response"]['groups'][0]['items']

    venues_list.append([(
     name,
     lat,
     lng,
     v['venue']['name'],
     v['venue']['location']['lat'],
     v['venue']['location']['lng'],
     v['venue']['categories'][0]['name']) for v in results])

Convert the list of venues to a dataframe.

In [6]:
Dublin_venues_df = pd.DataFrame(columns = ['Postcode', 'Postcode Latitude',
                             'Postcode Longitude', 'Venue',
                             'Venue Latitude', 'Venue Longitude',
                             'Venue Category'])
i =0
for venues in venues_list:
    for venue in venues:
        Dublin_venues_df.loc[i] = [venue[0], venue[1], venue[2], venue[3], venue[4], venue[5], venue[6]]
        i = i+1

Visualise the returned dataframe of venues

In [7]:
Dublin_venues_df.head(10)

,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Dublin 1,53.35,-6.26,Pull&Bear,53.349533,-6.262053,Clothing Store
1,Dublin 1,53.35,-6.26,Vice Coffee Inc.,53.347915,-6.262327,Coffee Shop
2,Dublin 1,53.35,-6.26,Dealz,53.350623,-6.263183,Discount Store
3,Dublin 1,53.35,-6.26,Gino's Gelato,53.349573,-6.261846,Ice Cream Shop
4,Dublin 1,53.35,-6.26,147 Deli,53.353410,-6.259807,Deli / Bodega
5,Dublin 1,53.35,-6.26,Holiday Inn Express Dublin City Centre Hotel,53.352182,-6.260848,Hotel
6,Dublin 1,53.35,-6.26,Abbey Theatre,53.348542,-6.257492,Theater
7,Dublin 1,53.35,-6.26,Offbeat Donut Co,53.347435,-6.255535,Donut Shop
8,Dublin 1,53.35,-6.26,101 Talbot,53.350287,-6.257196,Restaurant
9,Dublin 1,53.35,-6.26,The Celt,53.350442,-6.255071,Pub


In [8]:
Dublin_venues_df.shape

(901, 7)

Create a dataframe of one hot encoded venue categories.

In [9]:
# one hot encoding
Dublin_onehot = pd.get_dummies(Dublin_venues_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Dublin_onehot['Postcode'] = Dublin_venues_df['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [Dublin_onehot.columns[-1]] + list(Dublin_onehot.columns[:-1])
Dublin_onehot = Dublin_onehot[fixed_columns]

Dublin_onehot.head()

,Postcode,Advertising Agency,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baby Store,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio,Zoo,Zoo Exhibit
0,Dublin 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Dublin 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Dublin 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Dublin 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Dublin 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group the one hot encoded venue categories by postcode.

In [10]:
Dublin_grouped = Dublin_onehot.groupby('Postcode').mean().reset_index()
Dublin_grouped.head()

,Postcode,Advertising Agency,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baby Store,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio,Zoo,Zoo Exhibit
0,Dublin 1,0.0,0.0,0.0,0.01,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.01,0.0,0.01,0.01,0.00,0.0,0.0,0.0,0.0
1,Dublin 10,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
2,Dublin 11,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
3,Dublin 12,0.0,0.0,0.0,0.00,0.0,0.045455,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
4,Dublin 13,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.00,0.05,0.0,0.0,0.0,0.0


Cluster the grouped venue categories using 5 clusters

In [23]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 4

Dublin_grouped_clustering = Dublin_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Dublin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 1, 1, 1, 3, 1, 0, 1])

Define a function to find the top venue categoeries per postcode

In [12]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]                                                                                                                                                                                                                                                                                        

Implement the defined function and create a new dataframe of sorted venues by postcode, finding the most common 10 venues by postcode.

In [24]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


# create a new dataframe
postcode_venues_sorted = pd.DataFrame(columns=columns)
postcode_venues_sorted['Postcode'] = Dublin_grouped['Postcode']

for ind in np.arange(Dublin_grouped.shape[0]):
     postcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Dublin_grouped.iloc[ind, :], num_top_venues)

postcode_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Dublin 1,Pub,Café,Coffee Shop,Restaurant,Hotel,Italian Restaurant,Bookstore,Bakery,Clothing Store,Cocktail Bar
1,Dublin 10,Convenience Store,Golf Course,Hotel,Pub,Sports Club,Coffee Shop,Park,Supermarket,Go Kart Track,Food & Drink Shop
2,Dublin 11,Grocery Store,Pet Store,Pub,Coffee Shop,Pizza Place,Supermarket,Park,Bar,Food & Drink Shop,Flower Shop
3,Dublin 12,Supermarket,Coffee Shop,Chinese Restaurant,Grocery Store,Bar,Diner,Pizza Place,Shopping Mall,Fast Food Restaurant,Fish & Chips Shop
4,Dublin 13,Supermarket,Train Station,Burger Joint,Veterinarian,Fast Food Restaurant,Bar,Shopping Mall,Grocery Store,Badminton Court,Restaurant


Add the clustering labels and merge the most common venues with the original dataset.

In [25]:
#add clustering labels
postcode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Dublin_merged = DublinData

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Dublin_merged = Dublin_merged.join(postcode_venues_sorted.set_index('Postcode'), on='Postcode')

Dublin_merged.head() # check the last columns!

,District,Postcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Dublin,Dublin 1,53.35,-6.26,0,Pub,Café,Coffee Shop,Restaurant,Hotel,Italian Restaurant,Bookstore,Bakery,Clothing Store,Cocktail Bar
1,Dublin,Dublin 2,53.33,-6.25,0,Pub,Coffee Shop,Café,Hotel,Restaurant,Park,Cocktail Bar,French Restaurant,Indian Restaurant,Steakhouse
2,Dublin,Dublin 3,53.36,-6.22,0,Coffee Shop,Hotel,Pub,Café,Supermarket,Bar,Harbor / Marina,Fast Food Restaurant,Convenience Store,Restaurant
3,Dun Laoghaire Rathdown,Dublin 4,53.32,-6.23,0,Pub,Café,Restaurant,Coffee Shop,Hotel,Stadium,Grocery Store,Thai Restaurant,Indian Restaurant,French Restaurant
4,Dublin,Dublin 5,53.38,-6.19,0,Grocery Store,Convenience Store,Train Station,Pub,Gym,Pharmacy,Sports Club,Fast Food Restaurant,Pizza Place,Flower Shop


Visualise the results.

In [26]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Dublin_merged['Latitude'], Dublin_merged['Longitude'], Dublin_merged['Postcode'], Dublin_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=30,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7,
        parse_html=True).add_to(map_clusters)
       
map_clusters